In [1]:
# SETUP
#
# Install keras R
# install.packages('keras', repos = "https://cloud.r-project.org")
# 
# Update reticulate from cran (it defaults to mran which has an outdated version)
# install.packages("reticulate", repos = "https://cloud.r-project.org")

In [2]:
library(keras)
use_python('/anaconda/envs/py35')

# Import util functions
source("./common/utils.R")

# Import hyper-parameters
params <- load_params("inference")

Loading required package: rjson


In [3]:
# reticulate::py_config()

In [4]:
cat("OS:", Sys.info()["sysname"], "\n")
cat(R.version$version.string, "\n")
cat("Keras:", paste0(packageVersion("keras")), "\n")
cat("Tensorflow:", paste0(packageVersion("tensorflow")), "\n")
cat("Keras using", backend()$backend(), "\n")
cat("Keras channel ordering is", backend()$image_data_format(), "\n") 
cat("GPU: ", get_gpu_name(), "\n")
cat(get_cuda_version(), "\n")
cat(get_cudnn_version(), "\n")

OS: Linux 
R version 3.4.1 (2017-06-30) 
Keras: 2.1.5 
Tensorflow: 1.5 
Keras using tensorflow 
Keras channel ordering is channels_last 
GPU:  Tesla P100-PCIE-16GB 
CUDA Version 8.0.61 
CuDNN Version 6.0.21 


In [5]:
# Generate some fake data for inference
fake_data <- give_fake_data(batches = params$BATCH_SIZE * params$BATCHES_GPU, col_major = FALSE)

In [6]:
cat('x_train shape:', dim(fake_data), '\n')

x_train shape: 1280 224 224 3 


In [7]:
# Instantiate the model
model <- application_resnet50(include_top = FALSE, input_shape = c(224L, 224L, 3L))

In [8]:
summary(model)

________________________________________________________________________________
Layer (type)              Output Shape      Param #  Connected to               
input_1 (InputLayer)      (None, 224, 224,  0                                   
________________________________________________________________________________
conv1_pad (ZeroPadding2D) (None, 230, 230,  0        input_1[0][0]              
________________________________________________________________________________
conv1 (Conv2D)            (None, 112, 112,  9472     conv1_pad[0][0]            
________________________________________________________________________________
bn_conv1 (BatchNormalizat (None, 112, 112,  256      conv1[0][0]                
________________________________________________________________________________
activation_1 (Activation) (None, 112, 112,  0        bn_conv1[0][0]             
________________________________________________________________________________
max_pooling2d_1 (MaxPooli (N

In [9]:
t <- system.time(
    features <- predict(model, x = fake_data, batch_size = params$BATCH_SIZE)
)

t

   user  system elapsed 
  6.161   1.612   7.432 

In [10]:
# Time execution on channels-first data
print(paste("Images per second ", (params$BATCH_SIZE * params$BATCHES_GPU)/t[3]))

[1] "Images per second  172.228202368138"
